# Summary

+ Overview
    + efficientnet-b1,b2를 이용하여 학습을 진행했습니다.
    + arcface loss, mixup 등 시도에서 별다른 성능 향상을 가져오지는 않았습니다.
    + 다른 팀들과는 다르게 efficientnet-b6,b7에 대해서 성능이 좋지 않아서 사용하지 않았습니다.
    + 따라서, 기존에 학습했던 모델들을 이용해서 앙상블 및 후처리에 대하여 고민을 했습니다.
    + 첫번째로 클래스 불균형으로 인한 good의 과한 예측을 피하고자 하였습니다.
        1. 기본적으로 good인 이미지들은 어떠한 기하학적 변형을 해도 매우 확실하게 good이라고 예측할거라 가정했습니다.
        2. 너무 과하게 good으로 예측하는 경우를 조금이라도 완화시키고자 각 모델들 output에 softmax를 취한 후 앙상블했습니다..
        3. 비정상의 경우를 bad로 통일한 후 모델을 학습한 결과를 이용하여 후처리를 진행했습니다.
    + 두번째로 위의 결과를 이용해도 헷갈려하는 클래스들(pill, zipper, toothbrush, transistor, capsule)에 대해 추가 학습을 진행했습니다.
        1. 한 개의 클래스에 대해서만 학습을 한 후 하드보팅 또는 단일 모델의 결과를 가지고 후처리를 진행했습니다. 

+ Training
    + https://github.com/alswlsghd320/dacon-anomaly/blob/master/multi_train.sh에 들어가시면 자세한 학습 세팅을 아실 수 있습니다.

+ Inference & Post-processing
    + val loss가 가장 낮은 모델들 중 8개를 이용하여 TTA(rotate90 0, 90, 180, 270) 결과에 대해 softmax를 구한 후 npy형태로 저장했습니다.
    + 해당 npy파일들을 전부 불러와 평균을 취한 후 argmax를 취해 초기 예측값들을 구했습니다.
    + 그 후 비정상 클래스들을 (class)-bad로 수정하여 별개로 30개 클래스에 대해서 efficientnet-b4를 이용하여 5-fold 학습을 진행했습니다.
    + 위 5-fold good-bad 모델 예측에서 1) bad로 예측하거나 2) good으로 예측했지만 softmax값이 0.999999보다 작은 인덱스들을 추출했습니다.
    + 추출한 인덱스들 중 원래 예측값이 good으로 되어 있는 경우 해당 레이블이 아닌 2번째로 높았던 레이블로 예측하게 했습니다.
    + pill, zipper, toothbrush, transistor, capsule에 대해 각각 해당 레이블만 추출하여 추가로 학습을 진행했습니다.
    + toothbrush만 학습한 단일 모델을 이용하여 원래의 예측값이 아닌 해당 예측값들로 변경을 했습니다.
    + zipper는 원래의 예측값과 세 개의 zipper만 학습한 모델을 이용하여 하드보팅을 진행했습니다.
    + 나머지 세 클래스의 경우는 어떤 방법을 사용해도 성능 하락이 있어서 적용하지 않았습니다.

+ Summary
    + 각 단일 모델마다 TTA를 거친 후 softmax 계산된 결과를 저장 및 앙상블 예측을 진행했습니다.
    + good-bad 5-fold 앙상블 모델을 이용하여 덜 확신을 가지고 good이라고 예측한 레이블들을 변경했습니다.
    + 그럼에도 불구하고 헷갈려하는 클래스들에 대하여 해당 클래스만 따로 학습을 진행했습니다.
    + 그 결과 toothbrush는 단일 모델의 예측값으로 변경, zipper는 원래의 예측값과 세 개의 추가 모델들을 하드 보팅한 예측값으로 변경했습니다.


+ Score
    + effb1_384_img_size_aug_pillzipper 5fold ensemble => 0.8518
    + effb2_bestloss 3개 앙상블 => 0.8577
    + effb1_384_img_size 5fold ensemble => 0.8548
    + effb1_384_img_size 5fold ensemble + good-bad 후처리 적용 => 0.8729
    + effb2_bestloss 3개, effb1_384_img_size 5fold 앙상블 + good-bad 후처리 + toothbrush 후처리 => 0.8990
    + effb2_bestloss 3개, effb1_384_img_size 5fold 앙상블 + good-bad 후처리 + toothbrush,zipper 후처리 => 0.9016
    + effb1_384_img_size(0,4 fold), effb2_bestloss 4개, effb1_384_img_size_aug_pillzipper(0,4 fold) + good-bad 후처리 + toothbrush,zipper 후처리 => 0.9087

# Libaray

In [181]:
import os
import glob
from os.path import join as opj

import numpy as np
import pandas as pd 
from tqdm import tqdm
from easydict import EasyDict
from torch.cuda.amp import autocast
from sklearn.preprocessing import LabelEncoder
from collections import Counter

from dataloader import *
from network import *

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore')

# Setting

In [182]:
sub = pd.read_csv('files/sample_submission.csv')
bad_df = pd.read_csv('../data/train_df_bad.csv')  
 
le_bad = LabelEncoder() #le_bad
bad_df['label'] = le_bad.fit_transform(bad_df['label'])

good = le_bad.transform([label for label in le_bad.classes_ if 'good' in label]) #30개
ngood = le_bad.transform([label for label in le_bad.classes_ if not 'good' in label])

train_df = pd.read_csv('../data/train_df.csv')
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
good2 = le.transform([label for label in le.classes_ if 'good' in label]) # 88개

In [183]:
# 앙상블 예측 함수
def get_preds(li, good=good, ngood=ngood, good2=good2, le=le):
    ww = np.array([np.load(i) for i in li]) # 8 x 2514 x 30
    w = ww.mean(axis=0) # 2514 x 30
    w_maxs = np.max(w, axis=1)
    w_preds = np.argmax(w, axis=1)

    df_k2 = pd.DataFrame(data = w_maxs, columns=['max'])
    df_k2['preds'] = w_preds
    df_k2['label'] = le.inverse_transform(w_preds) #string

    bad2 = np.load('files/effb4_bad_5fold.npy') #2514 x 30

    bad2_maxs = np.max(bad2, axis=1)
    bad2_preds = np.argmax(bad2, axis=1)
    df_bad2 = pd.DataFrame(data = bad2_maxs, columns=['max'])
    df_bad2['preds'] = bad2_preds

    # good-bad에서 bad로 예측하거나 good으로 예측해도 softmax값이 0.999999보다 작은 인덱스들 추출
    idx2 = np.array(df_bad2[((df_bad2['preds'].isin(good)) & (df_bad2['max'] <0.999999)) | df_bad2['preds'].isin(ngood)].index)

    #위에서 구한 인덱스들 중에서 예측 레이블이 good인 경우면 2번째 높은 레이블로 변경
    idx_bad2 = np.array(df_k2.loc[idx2][df_k2['label'].isin(le.inverse_transform(good2))].index)
    p_bad2 = np.argsort(w, axis=1)[idx_bad2, -2]
    
    df_k2['label'].iloc[idx_bad2]= le.inverse_transform(p_bad2)
        
    return df_k2['label'].values

In [184]:
li = glob.glob('files/softmax_*.npy')
sub['label'] = get_preds(li, good=good, ngood=ngood, good2=good2, le=le)
sub.head()

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good


## Use One-class Classification model

In [185]:
def Postprocessing_oneclass(cls, sub, npys):
    df_sub = sub.copy()
    idxLst = [df_sub.iloc[idx]['index'] for idx in range(len(df_sub)) if cls in df_sub.iloc[idx]['label']]
    
    if not npys:
        raise AssertionError('npys must not be empty') 
    # 단일모델 예측 : 기존 모델 예측값 대신 단일 모델 예측값으로 전부 변경
    elif len(npys) == 1:
        path = npys[0]
        p = np.load(path, allow_pickle=True)
        df_sub.loc[idxLst,'label'] = p 

    # 하드보팅 예측 : 단일 모델들의 예측값과 원래의 예측값에 대하여 hard voting
    else:
        df = df_sub[df_sub['index'].isin(idxLst)]

        for path in npys:
            num = os.path.basename(path).split('.')[0][-3:]
            p = np.load(path, allow_pickle=True)
            df[f'pred_{num}'] = p
        
        for i in range(len(df)):
            label_pred_list = [df.iloc[i,1],df.iloc[i,2],df.iloc[i,3],df.iloc[i,4]]
            if (len(Counter(label_pred_list).most_common(2)) >1) and (Counter(label_pred_list).most_common(2)[1][1] == 2):
     
                newlabel = df_sub.loc[df.iloc[i]['index'],'label']
  
            else:
                newlabel = max(label_pred_list, key=label_pred_list.count)
            
            df_sub.loc[df.iloc[i]['index'],'label'] = newlabel
        
    return df_sub

In [186]:
sub1 = Postprocessing_oneclass('toothbrush', sub, ['files/toothbrush_220.npy','files/toothbrush_221.npy','files/toothbrush_222.npy'])
sub2 = Postprocessing_oneclass('zipper', sub1, ['files/zipper_254.npy', 'files/zipper_255.npy', 'files/zipper_256.npy'])

sub2.to_csv('./best_ensemble2.csv',index=False)


In [187]:

# sub1 = Postprocessing_oneclass('toothbrush', sub, ['files/toothbrush_220.npy'])
# sub2 = Postprocessing_oneclass('zipper', sub1, ['files/zipper_254.npy', 'files/zipper_255.npy', 'files/zipper_256.npy'])

# sub2.to_csv('./best_ensemble2_zipper_tooth.csv',index=False)